## Preprocessing

In [25]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [26]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])


In [27]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [28]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [29]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [30]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [31]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()

classification_counts_gt_1 = classification_counts[classification_counts > 1]
print(classification_counts_gt_1)


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [32]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000

classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [33]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, drop_first=True)



In [34]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']
# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
scaler.fit(X_train)

# Scale the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=X_train_scaled.shape[1], activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 80)                  │           2,960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,421 (21.18 KB)

 Trainable params: 5,421 (21.18 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [20]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7065 - loss: 0.5918 - val_accuracy: 0.7362 - val_loss: 0.5558
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7288 - loss: 0.5553 - val_accuracy: 0.7382 - val_loss: 0.5504
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7281 - loss: 0.5522 - val_accuracy: 0.7382 - val_loss: 0.5462
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7292 - loss: 0.5500 - val_accuracy: 0.7362 - val_loss: 0.5471
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7302 - loss: 0.5504 - val_accuracy: 0.7332 - val_loss: 0.5477
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7337 - loss: 0.5471 - val_accuracy: 0.7372 - val_loss: 0.5445
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7325 - loss: 0.5470 - val_accuracy: 0.7396 - val_loss: 0.5460
Epoch 8/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7348 - loss: 0.5405 - val_accu

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 1ms/step - accuracy: 0.7259 - loss: 0.5671
Loss: 0.5670852065086365, Accuracy: 0.7259474992752075


In [36]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')


they are neither targets nor features?**
   - The following variables were removed:
     - **`EIN`**: Identification column.
     - **`NAME`**: Organization name column.



In [38]:
from google.colab import files
files.download('AlphabetSoupCharity.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Report on Neural Network Model

## Overview of the Analysis
The purpose of this analysis was to create a binary classification model to predict whether applicants funded by Alphabet Soup Charity would be successful. The goal was to assist the charity in making informed funding decisions and maximizing the impact of its resources. The dataset provided contained information on more than 34,000 applications, including categorical and numerical variables such as application type, classification, requested funding amount, and more.

---

## Results

### **1. Data Preprocessing**
1. **What variable(s) are the target(s) for your model?**
   - The target variable is **`IS_SUCCESSFUL`**, which indicates whether funding was successful:
     - `1`: Successful
     - `0`: Not successful.

2. **What variable(s) are the features for your model?**
   - The features include all columns except `EIN`, `NAME`, and `IS_SUCCESSFUL`. After preprocessing, the following features were used:
     - `APPLICATION_TYPE`
     - `AFFILIATION`
     - `CLASSIFICATION`
     - `USE_CASE`
     - `ORGANIZATION`
     - `STATUS`
     - `INCOME_AMT`
     - `SPECIAL_CONSIDERATIONS`
     - `ASK_AMT`

3. **What variable(s) should be removed from the input data because they are neither targets nor features?**
   - The following columns were removed:
     - **`EIN`**: An identification column with no predictive value.
     - **`NAME`**: A name column with no predictive value.

4. **Preprocessing Steps:**
   - **Categorical Grouping**:
     - `APPLICATION_TYPE`: Categories with fewer than 528 occurrences were grouped into `"Other"`. The final counts for the top categories:
       - `T3`: 27,037
       - `T4`: 1,542
       - `T6`: 1,216
       - `T5`: 1,173
       - `T19`: 1,065
       - `Other`: 276
     - `CLASSIFICATION`: Categories with fewer than 1,883 occurrences were grouped into `"Other"`. The final counts for the top categories:
       - `C1000`: 17,326
       - `C2000`: 6,074
       - `C1200`: 4,837
       - `C3000`: 1,918
       - `C2100`: 1,883
       - `Other`: 2,261
   - **Scaling**:
     - `ASK_AMT`: Standardized using `StandardScaler` to bring it into a similar range as other features.
   - **Encoding**:
     - One-hot encoding was applied to categorical variables (e.g., `APPLICATION_TYPE`, `CLASSIFICATION`, `AFFILIATION`, etc.).
   - **Splitting the Data**:
     - The data was split into training (80%) and testing (20%) datasets.

---

### **2. Model Architecture**
1. **How many neurons, layers, and activation functions did you select for your neural network model, and why?**
   - The neural network was structured as follows:
     - **Input Layer**:
       - Number of input features: `<replace with number of features after one-hot encoding>`
     - **Hidden Layers**:
       - First Hidden Layer: 80 neurons, `ReLU` activation function.
       - Second Hidden Layer: 30 neurons, `ReLU` activation function.
     - **Output Layer**:
       - 1 neuron, `sigmoid` activation function (to output a probability for binary classification).
   - **Reasons for Choices**:
     - The `ReLU` activation function was selected for hidden layers as it efficiently handles non-linear patterns.
     - The number of neurons was chosen based on general best practices and the complexity of the data.
     - The `sigmoid` activation function was used for the output layer to handle binary classification.

2. **Model Summary**:
Model: "sequential" ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓ ┃ Layer (type) ┃ Output Shape ┃ Param # ┃ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩ │ dense (Dense) │ (None, 80) │ 2,960 │ │ dense_1 (Dense) │ (None, 30) │ 2,430 │ │ dense_2 (Dense) │ (None, 1) │ 31 │ └──────────────────────────────────────┴─────────────────────────────┴─────────────────┘ Total params: 5,421 Trainable params: 5,421 Non-trainable params: 0


3. **Were you able to achieve the target model performance?**
- The target accuracy was **75%**, but the model achieved the following results on the test dataset:
  - **Loss**: 0.5671
  - **Accuracy**: 72.59%
- While the performance was reasonable, it did not meet the target accuracy threshold of 75%.

---

### **3. Optimization Attempts**
**What steps did you take in your attempts to increase model performance?**

Several optimization strategies were tried to improve the model's performance:

1. **Increased Neurons**:
- Increased the number of neurons in the hidden layers to 100 (first layer) and 50 (second layer).
- **Result**: Minimal improvement in accuracy.

2. **Added a Third Hidden Layer**:
- Added an additional hidden layer with 25 neurons.
- **Result**: No significant change in accuracy.

3. **Increased Epochs**:
- Increased the number of training epochs from 100 to 150 to allow the model more time to learn.
- **Result**: The model began overfitting the training data, with no improvement in validation accuracy.

---

### **4. Summary**
1. **Overall Results**:
- The final deep learning model achieved a test accuracy of **72.59%**. While this is below the target accuracy of 75%, the model demonstrated its ability to learn patterns in the data.

2. **Limitations**:
- The dataset may contain features that are not highly predictive of success, which may have limited the model’s performance.
- Deep learning models may not be the best approach for tabular data with a high proportion of categorical variables.

3. **Alternative Models**:
- A **Random Forest Classifier** or **Gradient Boosting (e.g., XGBoost)** could be explored. These models are often more effective for structured/tabular data and can provide robust performance without extensive hyperparameter tuning.

4. **Recommendation**:
- While the current deep learning model provides reasonable accuracy, it is recommended to experiment with simpler machine learning models like Random Forest and Gradient Boosting to potentially achieve higher accuracy. Additionally, further feature engineering or collecting more data could enhance the model's predictive power.
